In [1]:
from russtress import Accent
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import pymorphy2
from gensim.models import KeyedVectors
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from copy import deepcopy

In [8]:
a = Accent()
morph = pymorphy2.MorphAnalyzer()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()
morph = pymorphy2.MorphAnalyzer()
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [355]:
with open (r'rusgram\Том _1\172-190_edited.html', 'r') as text:
    text = text.read()

In [356]:
soup = BeautifulSoup(text, markupMassage=False)
paragraphs = soup.find_all(['p','center'])

In [357]:
clean_text = []

In [358]:
for paragraph in paragraphs:
    if (paragraph.name == 'center')and(paragraph.b != None):
        paragraph_title = paragraph.find_all('b')
        paragraph_title = ["{}".format(x) for x in paragraph_title]
        paragraph_title = re.sub('<br/>', '\n', (' ').join(paragraph_title))
        paragraph_title = re.sub('<.+?>', '', paragraph_title)
        clean_text.append((paragraph_title, 'paragraph_title'))
    elif 'JUSTIFY' in paragraph.attrs.values():
        paragraph = re.sub('\xa0', ' ', str(paragraph))
        paragraph = re.sub('<i>', ' example ', paragraph)
        paragraph = re.sub('</i>', ' endexample ', paragraph)
        paragraph = re.sub('<span style="letter-spacing: 3px">', ' term ', paragraph)
        paragraph = re.sub('</span>', ' endterm ', paragraph)
        paragraph = re.sub('<.+?>', '', paragraph)
        clean_text.append((paragraph, 'paragraph'))

In [359]:
clean_text[0][0]

'ОСНОВНЫЕ ПОНЯТИЯ МОРФЕМИКИ. \nСЛОВООБРАЗОВАНИЕ\n'

In [360]:
def check_word(word, lang='ru'):
    # убираем этот знак, так как иногда он фигурирует в русских словах для обозначения аффикса j, не выражающегося на письме
    word = re.sub('j', '', word)
    rus=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    en=set('qwertyuiopasdfghjklzxcvbnm')
    if lang == 'ru':
        return not rus.isdisjoint(word.lower())
    elif lang == 'en':
        return not en.isdisjoint(word.lower())
    else:
        print('Допустимые значения lang: en или rus')

In [361]:
def check_gender(tags):
    tags = str(tags)
    if 'masc' in tags:
        return 'masc'
    elif 'femn' in tags:
        return 'femn'
    elif 'neut' in tags:
        return 'neut'

In [362]:
def check_number(tags):
    tags = str(tags)
    if 'sing' in tags:
        return 'sing'
    if 'plur' in tags:
        return 'plur'

Проблема: часто можно встретить слова, помеченные как примеры, перемешанными с основным текстом. Выносить одно слово из стихотворного текста как пример странно. 

Что делать: хочется иметь определенный лимит, например, если у нас встречается меньше n или равно слов-примеров подряд, оставлять их в тексте, но сохранять пометку example, так как подобные слова мы не можем заменять. Также, если у нас меньше или равно m слов основного текста между примерами, то включать слова основного текста в примеры. 

Идея: в тексте примеры обрамлены тегами example, endexample. Если это предложение, все слова будут внутри пары тегов, если перечисление, каждое слово будет внутри своей пары. Берем каждый абзац и выполняем split по началу каждого примера (тег ' example '). В итоге можем получить три вида кусков:
* "основной текст"
* "пример" + ' endexample '
* "пример" + ' endexample ' + "основной текст"

Учитывая то, что если подстрока, по которой производится деление, стоит в конце или в начале, split все равно выдаст 2 элемента, можно легко выявлять куски первого типа и помечать их как paragraph.

In [363]:
'слово  endexample '.split(' example ')

['слово  endexample ']

In [364]:
'слово '.split(' example ')

['слово ']

Далее мы получаем куски, которые после split'а по подстроке ' endexample ' слева содержат пример, а справа — основной текст. При этом мы знаем, что далее за основным текстом был либо конец абзаца, либо начинался следующий пример.

Сделаем два допущения:
* текст начала параграфа, если это не пример, всегда считаем достаточно длинным, чтобы включить его в основной текст
* недостаточно длинный текст в конце параграфа, если он не является примером, включаем в пример

Дальнейшая логика в коде.

In [365]:
def check_sides(text):
    end_example = re.search('(^[^а-яА-ЯЁё]+?)[а-яА-ЯЁё]', text)
    if end_example:
        end_example = end_example.group(1)
    else:
        end_example = ''
    start_example = re.search('\s[^а-яА-ЯЁё]+?$', text)
    if start_example:
        start_example = start_example.group(0)
    else:
        start_example = ''
    length = len(text)
    length_end = len(end_example)
    length_start = len(start_example)
    text = text[length_end: length - length_start]
    return end_example, start_example, text

In [366]:
def extract_examples(paragraph, n=2, m=1):
    parts = paragraph.split(' example ')
    splitted = []
    example_line = ''
    
    for index, part in enumerate(parts):
        if part == '':  # параграф начинался с примера и первый элемент получился пустой строкой
            continue
        part = part.split(' endexample ')
        if len(part) == 1:  # параграф начинался не с примера, вписываем основной текст
            # проверим пунктуацию в конце текста
            end_example, start_example, part[0] = check_sides(part[0])
            splitted.append((part[0], 'paragraph'))
            example_line += start_example
        else:
            example_line += part[0]
            example_words = sum(1 for i in word_tokenize(example_line) if check_word(i))  # сумма идущих подряд слов-примеров
            text_words = sum(1 for i in word_tokenize(part[1]) if check_word(i))  # считаем количество слов в основном тексте
            if text_words <= m:
                # добавляем основной текст в примеры, если его мало
                # как вариант, слов основного текста может не быть вообще, но в переменной будет лежать пунктуация
                example_line += part[1]
            elif (example_words > n)and(text_words > m):
                # и примеров, и основного текста много, примеры записываем как примеры, текст — как текст
                # проверим пунктуацию по краям основного текста, потому что теги примеров обычно оставляют ее снаружи
                end_example, start_example, part[1] = check_sides(part[1])
                splitted.append((example_line + end_example, 'example'))
                example_line = start_example   
                splitted.append((part[1], 'paragraph'))
            elif (example_words <= n)and(text_words > m):
                # примеров мало, основного текста много, все записываем как текст
                # проверим пунктуацию в конце текста
                end_example, start_example, part[1] = check_sides(part[1])
                text = ' example ' + example_line + ' endexample ' + end_example + part[1]
                example_line = start_example  
                splitted.append((text, 'paragraph'))

    if example_line != '':  # если не обнулены примеры, вписываем
        splitted.append((example_line, 'example'))

    return splitted

In [367]:
extract_examples(clean_text[83][0])

[('Добавление постфиксального морфа дает еще  28  типов морфного строения словоформ:',
  'paragraph'),
 (' R-pt (где-то); pr-R-pt (от-куда-то); R-s-pt (тряс-я-сь); R-s-s-pt (толк-ну-вши-сь); R-s-s-s-pt (горб-ат-и-вшись); pr-R-pt (ис-пек-ши-сь); pr-R-s-s-pt (о-глядыва|j-а|-сь); pr-R-s-s-s-pt (на-брод-яж-нича-вши-сь); pr-pr-R-s-pt (при-от-кры-вши-сь); pr-pr-R-s-s-pt (при-от-кры-ва|j-а|-сь); pr-pr-R-s-s-s-pt (по-из-гряз-ни-вши-сь); pr-pr-pr-R-s-pt (по-при-от-кры-вши-сь); pr-pr-pr-R-s-s-pt (по-при-от-кры-ва-вши-сь); R-f-pt (бер-ут-ся, как-ой-либо); pr-R-f-pt (за-тряс-ут-ся, при-сол-и-те); pr-pr-R-f-pt (при-от-кро|j-у|т-ся); prpr-pr-R-f-pt (по-при-от-кро|j-у|т-ся); R-s-f-pt (толк-н-ут-ся); R-s-s-f-pt (горд-и-вш-ий-ся); R-s-s-sf-pt (кудр-яв-и-вш-ий-ся); pr-R-s-f-pt (при-крыва|j-y|т-ся); pr-R-s-s-f-pt (рас-красн-е-вш-ий-ся, пере-шуч-ива-л-и-сь); pr-R-s-s-s-f-pt (рас-кудр-яв-и-вший-ся); pr-pr-R-s-f-pt (при-на-кры-вш-ий-ся); pr-pr-Rs-s-f-pt (пере-на-сыт-и-вш-ий-ся); pr-pr-R-s-s-s-f-pt (по-из-гр

In [368]:
def preprocess(text):
                    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    new_text =[]
    flag = ''
    for info in doc.tokens:
        word = info.text
        pos = info.pos
        
        if word not in ['example', 'endexample', 'term', 'endterm']: # термины и примеры определяются по тегам
            pass
        elif word == 'example':
            flag = 'example'
            continue
        elif word == 'endexample':
            flag = ''
            continue
        elif word == 'term':
            flag == 'term'
            continue
        elif word == 'endterm':
            flag == ''
            continue

        if (flag == '')and(check_word(word)):
            word = (word, 'word', pos)
        elif (flag == '')and(check_word(word, lang='en')):
            word = (word, 'f_word', pos)
        elif (flag == 'example')and(check_word(word)):
            word = (word, 'example', pos)
        elif (flag == 'term')and(check_word(word)):
            word = (word, 'term', pos)
        else:
            word = (word, 'punct', pos)

        new_text.append(word)

    return new_text

* example - пример
* term - термин
* word - слово
* f_word - слово, содержащее латинские буквы, но не кириллицу (не термин и не пример)
* punct - пунктуация, цифры, приписанные не буквами

Соединим некоторые служебные слова (отрицательные частицы и непроизводные предлоги) со словами, за которыми они следуют.

In [369]:
def add_functional_words(paragraph):
    functional_words = ['ни', 'не', 'в', 'с', 'к', 'за', 'из-под', 'из-за', 'по-над', 'у', 'над',
                        'на', 'под', 'над', 'без', 'до', 'о', 'об', 'от', 'при', 'для', 'из', 'по']
    new_paragraph = []
    add = ''
    for word in paragraph:
        if word[0] in functional_words:
            add = word[0] + ' '
            continue
        if add != '':
            text = add + word[0]
            word = (text, word[1], word[2])
            new_paragraph.append(word)
            add = ''
        else:
            new_paragraph.append(word)
    return new_paragraph

In [370]:
def final_preprocess(text):
    text = extract_examples(text)
    new_text = []
    paragraph = []
    for part in text:
        if part[1] == 'paragraph':
            paragraph.extend(preprocess(part[0]))
        else:
            paragraph = add_functional_words(paragraph)
            new_text.append((paragraph, 'paragraph'))
            paragraph = []
            new_text.append(part)
    if paragraph != []:
        paragraph = add_functional_words(paragraph)
        new_text.append((paragraph, 'paragraph'))
    return new_text

In [371]:
final_preprocess(clean_text[7][0])

[([('Например', 'word', 'ADV'),
   (',', 'punct', 'PUNCT'),
   ('предложение', 'word', 'NOUN')],
  'paragraph'),
 ('Дом стоит на горе ', 'example'),
 ([('состоит', 'word', 'VERB'),
   ('из четырех', 'word', 'NUM'),
   ('словоформ', 'word', 'NOUN'),
   (':', 'punct', 'PUNCT')],
  'paragraph'),
 (' 1) дом, 2) стоит, 3) на, 4) горе; ', 'example'),
 ([('каждая', 'word', 'DET'),
   ('из них', 'word', 'PRON'),
   ('имеет', 'word', 'VERB'),
   ('определенное', 'word', 'ADJ'),
   ('значение', 'word', 'NOUN'),
   (',', 'punct', 'PUNCT'),
   ('и', 'word', 'CCONJ'),
   ('при этом', 'word', 'PRON'),
   ('отрезки', 'word', 'NOUN')],
  'paragraph'),
 ('дом, стоит и на горе ', 'example'),
 ([('свободно', 'word', 'ADV'),
   ('перемещаются', 'word', 'VERB'),
   (':', 'punct', 'PUNCT')],
  'paragraph'),
 ('на горе стоит дом; стоит дом на горе ', 'example'),
 ([('и', 'word', 'CCONJ'),
   ('тому', 'word', 'PRON'),
   ('подобное', 'word', 'ADJ'),
   (',', 'punct', 'PUNCT'),
   ('а', 'word', 'CCONJ'),
   ('

In [372]:
def count(word): #подсчет ударного слога и количества слогов в слове
    word = word.split(' ') # не хотим, чтобы ударение проставлялась в предлогах с двумя слогами
    if len(word) != 1:
        funcs = ' '.join(word[:len(word) - 1])
        word = funcs + ' ' + a.put_stress(word[-1])
    else:
        word = a.put_stress(word[0])
    functional_words = ['ни', 'не', 'в', 'с', 'к', 'за', 'из-под', 'из-за', 'по-над', 'у', 'над',
                        'на', 'под', 'над', 'без', 'до', 'о', 'об', 'от', 'ну', 'о', 'и', 'а', 'при',
                        'для', 'из', 'по', 'но', 'же']
    vowels = 'иёуеыаоэяю'
    if word not in functional_words: #если слово служебное, в нем не считается ударение
        stressed = word.split("'")[0]
        stressed = sum(1 for c in stressed if c in vowels)
    else:
        stressed = 0
    syllables = sum(1 for c in word if c in vowels)
    return stressed, syllables

In [373]:
def count_line(line): #подсчет ударных слогов и количества слогов в строке
    stresses_line = []
    syllables_line = 0
    for word in line:
        if word[1] in ['word', 'term', 'example']:
            stressed, syllables = count(word[0])
            if stressed != 0:
                stresses_line.append(syllables_line + stressed)
            syllables_line += syllables
    return stresses_line, syllables_line

In [374]:
def syllables_in_line(syllables_line):
    if syllables_line < 12:#если недостаточное количество слогов, продолжим набирать слова в строку
        return syllables_line, 'too few syllables'
    elif syllables_line > 17:#если слишком много слогов, меняем строку
        return syllables_line, 'too many syllables'
    else:
        return syllables_line, 'syllables true'

In [375]:
def first_stress(stresses_line):
    if stresses_line[0] not in [1, 3, 4]:#если первое ударение не стоит на 1, 3 или 4 слоге, меняем строку
        return stresses_line[0], 'the first stress does not suit'
    else:
        return stresses_line[0], 'first stress true'

In [376]:
def last_stress(stresses_line, syllables_line):
    if (syllables_line - stresses_line[-1]) != 1:#если последнее ударение не на предпоследний слог, меняем строку
        return (syllables_line - stresses_line[-1]), 'the last stress does not suit'
    else:
        return (syllables_line - stresses_line[-1]), 'last stress true'

In [419]:
def stresses(stresses_line, syllables_line):
    if stresses_line[0] == 1:#в зависимости от наличия пропущенного ударного слога в начале, присваеваем кол-во слогов
        steps = 1 #steps - ударные слоги, в том числе пропущенные
    else:
        steps = 2
    checker = False #инициализируем значение флага: правильность кол-ва слогов между ударными слогами
    for index, num in enumerate(stresses_line[-1:0:-1]):#перебираем индексы ударных слогов от последнего до второго включительно
        index = (index + 1)*-1 #index counted from end
        step = num - stresses_line[index -1] - 1 #количество слогов между текущим ударным слогом и предыдущим
        if step in [1, 2, 3, 4, 5]:
            checker = True
            if step in [3, 4, 5]:#в зависимости от наличия "пропущенного" ударного слога, присваеваем кол-во слогов
                steps += 2
            else:
                steps += 1
        else:
            checker = False
            return index, 'wrong number of syllables between stresses' #между ударением на этом индексе и предыдущим неправильное кол-во слогов
    if (checker == True)and(steps == 6):
        return 0, 'true'
    else:
        return steps, 'wrong number of stresses'

In [378]:
def check_line(stresses_line, syllables_line):
    # общая проверка
    num, message = syllables_in_line(syllables_line)
    if message != 'syllables true':
        return num, message
    num, message = first_stress(stresses_line)
    if message != 'first stress true':
        return num, message
    num, message = last_stress(stresses_line, syllables_line)
    if message != 'last stress true':
        return num, message
    num, message = stress(stresses_line, syllables_line)
    if message != 'true':
        return num, message

In [379]:
def flection(lex_neighb, tags):
    tags = str(tags)
    tags = re.sub(',[AGQSPMa-z-]+? ', ',', tags)
    tags = tags.replace("impf,", "")
    tags = re.sub('([A-Z]) (plur|masc|femn|neut|inan)', '\\1,\\2', tags)
    tags = tags.replace("Impe neut", "")
    tags = tags.split(',')
    tags_clean = []
    for t in tags:
        if t:
            if ' ' in t:
                t1, t2 = t.split(' ')
                t = t2
            tags_clean.append(t)
    tags = frozenset(tags_clean)
    prep_for_gen = morph.parse(lex_neighb)
    ana_array = []
    for ana in prep_for_gen:
        if ana.normal_form == lex_neighb:
            ana_array.append(ana)
    for ana in ana_array:
        try:
            flect = ana.inflect(tags)
        except:
            print(tags)
            return None
        if flect:
            word_to_replace = flect.word
            return word_to_replace
    return None

Универсальный список POS-тегов, используется в natasha и word2vec'е: https://universaldependencies.org/u/pos/all.html

!Важно: прежде мы "приклеивали" предлоги и отрицательные частицы к словам. Необходимо учитывать это при замене.

In [380]:
def change_word(word, pos):
    word = word.split(' ')
    if len(word) != 1:
        func = ' '.join(word[:len(word) - 1]) + ' '
        word = word[-1]
    else:
        func = ''
        word = word[0]
    change = []
    parsed_word = morph.parse(word)[0]
    tags = parsed_word.tag
    normal_form = parsed_word.normal_form + '_' + pos
    if normal_form in model.vocab:
        vector = model[normal_form]
    else:
        return change
    synonyms = model.most_similar(positive=[vector], negative=[], topn=3)
    synonyms = [i[0].split('_') for i in synonyms[1:]]
    for synonym, pos_synonym in synonyms:
        if pos == pos_synonym:
            if pos == 'NOUN':
                number = check_number(tags)
                gender = check_gender(tags)
                gender_synonym = check_gender(morph.parse(synonym)[0].tag)
                if (number == 'sing')and(gender != gender_synonym):
                    continue
            if pos not in ['ADV', 'ADP', 'CCONJ', 'SCONJ', 'INTJ', 'PART']: # неизменяемые части речи
                synonym = flection(synonym, tags)
            if synonym:
                change.append(func + synonym)
    return change

# Типы ошибок:

### too few syllables

Исправление: Продолжаем собирать слова.

Проблема 1: далее следует кусок с примерами / конец текста.

Проблема 2: конец параграфа. Стоит ли оставлять деление на абзацы или объединить их, чтобы снизить количество таких ошибок?

Пока что при недостатке слов печатается None

### too many syllables: 

Такая ситуация возникает в случае, когда в строке n слов, и на n - 1 слове минимальное количество слогов (12) не было набрано, а на n-ом слове превысило лимит (17). Это значит, что последнее слово оказалось слишком длинным (минимум 7 слогов).

Исправление: укорачивать строку будем с помощью перестановки и/или замены. Перестановка: Попробуем поменять местами последнее слово со следующим, предполагая, что раз последнее слово оказалось достаточно длинным, следующее таким не окажется. Делать это будем только в том случае, если:

* между словами нет пунктуации
* текущее слово не пример
* следующее слово не пример
* следующее слово больше 1 слога (попытаемся избежать перемешивания союзов, непроизводных предлогов)
* следующее слово короче по количеству слогов
* разница между количеством слогов в нынешнем и следующем словах находится в промежутке \[k, k + 6\), где k — количество лишних слогов

Если этот метод не помогает, переходим к замене. Попробуем подвергнуть преобразованию два варианта строки, без перестановки и с, но только в случае, если перестановка не получилась, так как разница в количестве слогов не попала в нужный интервал. Приоритет отдается оригинальной строке: если она после замены стала подходящей, строка с перестановкой не затрагивается.

Логичнее всего заменять самое длинное слово, однако оно не должно быть ни примером, ни термином. Составим список таких слов, упорядоченных по количеству слогов, по убыванию. Сохраним с каждым словом индекс в оригинальной строке. Посчитаем количество слогов, от которого нам нужно избавиться: s - 17, где s — количество слогов в текущей строке.

Далее необходимо найти синонимы. Учитываются следующие факторы:

* род существительных в единственном числе при замене должен совпадать
* можно брать только первые два слова, не считая оригинального, предложенных моделью для замены
* части речи должны совпадать

Дело в том, что с возрастанием косинусного расстояния сильно страдает смысл, а "сэкономить" слоги мы можем, заменив несколько слов в строке.

Далее, каждый синоним нужно поставить в нужную форму. Если поставить в нужную форму не получается, считается, что синоним не найден.

Происходит перебор упорядоченных ранее слов, внутри цикла — перебор их синонимов. Каждый раз, когда находится более короткое слово, пересчитывается число лишних слогов. Как только число лишних слогов меньше или равно нулю, а общее количество слогов в строке не меньше 12, строка считается правильной.

Если ничего не поможет, вернется None и программа выдаст ошибку.

### syllables true:

!Важно: прежде мы сделали допущение, что в тексте могут присутствовать примеры, не более двух подряд. Если на конце сформированной по длине строки есть пример и следующее слово тоже является примером, вмещаем их в одну строку. Если длина строки будет слишком большой, проблема будет решаться функцией too_many_syllables методом замены. Помимо этого, в строку добавляются знаки препинания.

In [381]:
def too_many_syllables(line, num, num_syllables, preprocessed_text):
    redundant = num_syllables - 17
    if num + 1 != len(preprocessed_text):  # если текущее слово оказалось последним в подаваемом тексте
        last_word = preprocessed_text[num]
        next_word = preprocessed_text[num + 1]
        length = count(last_word[0])[1]
        length_next = count(next_word[0])[1]
        dif = length - length_next
        if (last_word[1] != 'example')and(next_word[1] != 'example')and(length_next > 1)and(redundant <= dif < redundant+6):
            last_word = line[-1]
            line[-1] = next_word
            preprocessed_text[num + 1] = last_word
            # перестановка удалась
            return line
        elif (last_word[1] != 'example')and(next_word[1] != 'example')and(length_next > 1):
            alternative = deepcopy(line)
            alternative[-1] = next_word
            choose_line = [line, alternative]
        else:
            choose_line = [line,]
    else:
        choose_line = [line,]
                    
    for i, line in enumerate(choose_line):
        redundant = num_syllables - 17
        target_words = [(w[0], w[2], count(w[0])[1], num) for num, w in enumerate(line) if w[1] == 'word']
        target_words = sorted(target_words, key=lambda x:x[2], reverse=True)
        for w, pos, length, index in target_words:
            synonyms = change_word(w, pos)
            if synonyms:  # синонимы могут не найтись
                synonyms = [(s, count(s)[1]) for s in synonyms]
                min_synonym = sorted(synonyms, key=lambda x:x[1])[0]
                dif = length - min_synonym[1]
                if dif > 0:
                    redundant -= dif
                    line[index] = (min_synonym[0], 'word', pos)
                if redundant <= 0:
                    break
        if (11 < redundant <= 0)and(i == 0):
            # замена в оригинальной строке
            return line
        elif (11 < redundant <= 0)and(i == 1):
            preprocessed_text[num + 1] = last_word
            # замена в строке с перестановкой
            return line

### the first stress does not suit

Исправление: находим первое по порядку слово, в котором ударение падает на 1, 3 или 4 слог. Передвигаем его в самое начало. Если передвигаемое слово является примером, за ним следует еще один пример и между ними нет знаков препинания, двигаем примеры вместе.

Если подходящих слов не нашлось, переходим к замене. Составляем список слов на замену: все они должны быть с пометкой word. Для каждого кандидата подбираем возможные синонимы. Перебираем синонимы и проверяем на два условия:

* ударение падает на 1, 3 или 4 слог
* общее количество слогов в строке находится в диапазоне \[12, 17\]

Первый подошедший синоним подставляем в начало строки, удалив оригинальное слово.

В случае, если ничего не поможет, вернется None и программа выдаст ошибку.

In [382]:
def the_first_stress_does_not_suit(line, syllables_line):
    line_check = deepcopy(line)
    target_words = [(num, w) for num, w in enumerate(line) if count(w[0])[0] in [1, 3, 4]]
    if len(target_words) != 0:
        # учитываем, что до этого мы сделали допущение: внутри текста могут идти только !!!два!!! слова-примера подряд
        target_index, target_word = target_words[0]
        if target_word[1] == 'example' and line[target_index + 1][1] == 'example':
            words_to_pop = [(target_index + 1, line[target_index + 1]), (target_index, target_word)]
        else:
            words_to_pop = [(target_index, target_word),]
        [line.pop(index) for index, w in words_to_pop]
        line.reverse()
        [line.append(w) for index, w in words_to_pop]
        line.reverse()
        return line
    else:
        target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word']
        for index, word, synonyms in target_words:
            syllables_word = count(word)[1]
            for synonym in synonyms:
                stressed, syllables_synonym = count(synonym)
                if stressed in [1, 3, 4] and 12 <= (syllables_line - syllables_word + syllables_synonym) <= 17:
                    line.pop(index)
                    line.reverse()
                    line.append((synonym, word[1], word[2]))
                    line.reverse()
                    break
        if line == line_check:
            return None
        else:
            return line

### the last stress does not suit
Исправление: находим первое с конца слово, ударение в котором падает на предпоследний слог и двигаем его в конец. Если слово является примером и перед ним шел другой пример, передвигаем их вместе.

Если подходящих слов не нашлось, переходим к замене. Составляем список слов на замену: все они должны быть с пометкой word. Для каждого кандидата подбираем возможные синонимы. Перебираем синонимы и проверяем на два условия:

* ударение падает на предпоследний слог
* общее количество слогов в строке находится в диапазоне \[12, 17\]

Первый подошедший синоним подставляем в начало конец, удалив оригинальное слово.

В случае, если ничего не поможет, вернется None и программа выдаст ошибку.

In [383]:
def the_last_stress_does_not_suit(line, syllables_line):
    line_check = deepcopy(line)
    target_words = [(num, w) for num, w in enumerate(line) if (count(w[0])[1] - count(w[0])[0]) == 1]
    if len(target_words) != 0:
        target_words.reverse()
        # учитываем, что до этого мы сделали допущение: внутри текста могут идти только !!!два!!! слова-примера подряд
        target_index, target_word = target_words[0]
        if target_word[1] == 'example' and line[target_index - 1][1] == 'example':
            words_to_pop = [ (target_index, target_word), (target_index - 1, line[target_index - 1])]
        else:
            words_to_pop = [(target_index, target_word),]
        [line.pop(index) for index, w in words_to_pop]
        words_to_pop.reverse()
        [line.append(w) for index, w in words_to_pop]
        return line
    else:
        target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word']
        target_words.reverse()
        for index, word, synonyms in target_words:
            syllables_word = count(word)[1]
            for synonym in synonyms:
                stressed, syllables_synonym = count(synonym)
                if (syllables_synonym - stressed) == 1 and 12 <= (syllables_line - syllables_word + syllables_synonym) <= 17:
                    line.pop(index)
                    line.append((synonym, word[1], word[2]))
                    break
        if line == line_check:
            return None
        else:
            return line

Исправление ошибок 4-ой проверки еще не прописано. На данный момент функция печатает None, если в строке недостаточно слов и их невозможно набрать, ломается при остальных ошибках 1, 2 и 3 проверок, если их невозможно исправить, просто переходит к набору следующей строки, если встречаются ошибки 4 проверки или строка является полностью верной.

### 4 проверка, примерный подход

### wrong number of syllables between stresses

Перестановки с подсчетом слогов между ударениями, при неудаче подключить замену. Пересмотр функции проверки stresses: стоит ли допустить пропуск не только 1, но и 2 ударных слогов? Количество слогов между ударными слогами возрастет с 5 максимально допустимых до 8.

### wrong number of stresses

Если получена эта ошибка, значит, все слова соответствуют назначенным критериям, кроме количества слогов в строке. Два случая:
* слогов меньше, чем нужно: можем осуществить вставку в конец так, чтобы добрать необходимое количество слогов. Последнее ударение должно падать на предпоследний слог
* слогов больше, чем нужно: поправить на этапе набора слов: невозможно использовать более 6 самостоятельных слов в строке? замена длинных слов на более короткие?

In [420]:
def rewrite(preprocessed_text):
    text_length = len(preprocessed_text)
    lines = []
    line = []
    for index, word in enumerate(preprocessed_text):
        line.append(word)
            
        stresses_line, syllables_line = count_line(line)
        # функция, поймавшая ошибку, передала информацию о количестве слогов в строке
        num_syllables, message = syllables_in_line(syllables_line)  # проверка 1/4
        if message == 'too few syllables':
            print(message, [w[0] for w in line])
            continue
        elif message == 'too many syllables':
            print(message, [w[0] for w in line])
            line = too_many_syllables(line, index, num_syllables, preprocessed_text)
        elif message == 'syllables true':
            print(message, [w[0] for w in line])
            if ((text_length != index + 1)and((preprocessed_text[index][1] == 'example' and preprocessed_text[index + 1][1] == 'example')
                or(preprocessed_text[index + 1][1] == 'punct'))):
                print('more examples')
                continue
        
        stresses_line, syllables_line = count_line(line)
        # функция, поймавшая ошибку, передала информацию о расположении первого ударения
        stress, message = first_stress(stresses_line) # проверка 2/4
        if message == 'the first stress does not suit':
            print(message, [w[0] for w in line])
            line = the_first_stress_does_not_suit(line, syllables_line)
            print([w[0] for w in line])
        elif message == 'first stress true':
            print([w[0] for w in line])
        
        stresses_line, syllables_line = count_line(line)
        # функция, поймавшая ошибку, передала информацию о расположении последнего ударения
        stress, message = last_stress(stresses_line, syllables_line) # проверка 3/4
        if message == 'the last stress does not suit':
            print(message, [w[0] for w in line])
            line = the_last_stress_does_not_suit(line, syllables_line)
            print([w[0] for w in line])
        elif message == 'last stress true':
            print(message, [w[0] for w in line])
        
        stresses_line, syllables_line = count_line(line)
        info, message = stresses(stresses_line, syllables_line)
        if message == 'wrong number of syllables between stresses':
            print(message, [w[0] for w in line])
            line = []
        elif message == 'wrong number of stresses':
            print(message, [w[0] for w in line])
            line = []
        elif message == 'true':
            print(message, [w[0] for w in line])
            line = []

In [421]:
clean_text[7][0]

'   Например, предложение  example Дом endexample   example стоит endexample   example на endexample   example горе endexample  состоит из четырех словоформ: 1)  example дом endexample , 2)  example стоит endexample , 3)  example на endexample , 4)  example горе endexample ; каждая из них имеет определенное значение, и при этом отрезки  example дом endexample ,  example стоит endexample  и  example на endexample   example горе endexample  свободно перемещаются:  example на endexample   example горе endexample   example стоит endexample   example дом endexample ;  example стоит endexample   example дом endexample   example на endexample   example горе endexample  и тому подобное, а отрезок  example на endexample   example горе endexample  может включать внутрь себя другие отрезки, характеризующиеся первым свойством ( example на endexample   example высокой endexample   example горе endexample ,  example на endexample   example очень endexample   example большой endexample   example горе

In [422]:
new_text = final_preprocess(clean_text[7][0])

In [423]:
for part, title in new_text:
    if title == 'paragraph':
        print(rewrite(part))
    else:
        print(part)

too few syllables ['Например']
too few syllables ['Например', ',']
too few syllables ['Например', ',', 'предложение']
None
Дом стоит на горе 
too few syllables ['состоит']
too few syllables ['состоит', 'из четырех']
too few syllables ['состоит', 'из четырех', 'словоформ']
too few syllables ['состоит', 'из четырех', 'словоформ', ':']
None
 1) дом, 2) стоит, 3) на, 4) горе; 
too few syllables ['каждая']
too few syllables ['каждая', 'из них']
too few syllables ['каждая', 'из них', 'имеет']
syllables true ['каждая', 'из них', 'имеет', 'определенное']
['каждая', 'из них', 'имеет', 'определенное']
the last stress does not suit ['каждая', 'из них', 'имеет', 'определенное']
['каждая', 'из них', 'определенное', 'имеет']
true ['каждая', 'из них', 'определенное', 'имеет']
too few syllables ['значение']
too few syllables ['значение', ',']
too few syllables ['значение', ',', 'и']
too few syllables ['значение', ',', 'и', 'при этом']
too few syllables ['значение', ',', 'и', 'при этом', 'отрезки']
Non

### Возможные вставки:

* к примеру
    * при отсутствии "например", "к примеру" в тексте
    * перед примерами

### Ошибки:

* неправильно расставленное ударение: гора/горе, не исправляется контекстуально
* при серьезных перестановках страдает пунктуация. Стоит подумать над ограничениями по пунктуации (двигаться в пределах отрезков, отделенных точкой/запятой, лепить запятые к слову, которое ими окружено, так как велика вероятность того, что оно вводное)
* (см. § N) не к месту, возможно, стоит выносить в конец сформированной строки, в которой встретилось указание.

In [189]:
a.put_stress('на высокой горе')

"на высо'кой го'ре"